In [28]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from hyperion.model import ModelOutput
import astropy.constants as const
from photutils import aperture_photometry as ap
from photutils import CircularAperture

In [19]:
pc = const.pc.cgs.value

In [20]:
m = ModelOutput('/Users/yaolun/test/model86.rtout')
image = m.get_image(group=5, inclination=0, distance=178. * pc, units='MJy/sr')

In [35]:
# The radius of the aperture in arcsec
radius = 10
aper_size = np.pi*radius**2 / 4.25e10 # in sr
# The offset to the center
offset = 10

### IRAC band (3.6, 4.5, 5.8, 8.0 $\mu$m)

In [39]:
irac_wl = [3.6, 4.5, 5.8, 8.0]
irac_wl = [3.6]
# fig = plt.figure(figsize=(8,6))
# ax = fig.add_subplot(111)
for wl in irac_wl:
    iwav = np.argmin(np.abs(wl - image.wav))
    # Image in the unit of MJy/sr, change it into erg/s/cm2/Hz/sr
    factor = 1e-23*1e6
    val = image.val[::-1, :, iwav] * factor + 1e-30
    
    # Calculate the image width in arcseconds given the distance used above
    # get the max radius
    rmax = max(m.get_quantities().r_wall)
    w = np.degrees(rmax / image.distance) * 3600.
    
    pos_n = (len(val[0,:])/2.-1 + offset*len(val[0,:])/2/w,len(val[0,:])/2.-1)
    pos_s = (len(val[0,:])/2.-1 - offset*len(val[0,:])/2/w,len(val[0,:])/2.-1)
    
    aper_n = CircularAperture(pos_n, r=radius * len(val[0,:])/2/w )
    aper_s = CircularAperture(pos_s, r=radius * len(val[0,:])/2/w )
    # multiply the aperture size in sr and convert to Jy
    phot_n = ap(val, aper_n)['aperture_sum'].row * aper_size * 1e23
    phot_s = ap(val, aper_s)['aperture_sum'] * aper_size * 1e23
    print phot_n, phot_s

AttributeError: 'Column' object has no attribute 'row'